# Lesson 3: Mastering SQS Operations: Sending, Receiving, and Deleting Messages


Today's lesson delves into the fundamentals of interacting with Amazon's Simple Queue Service (SQS) using the boto3 library in Python, focusing specifically on Standard queues and FIFO (First-In, First-Out) queues. We'll explore the nuances of sending messages to both queue types, receiving messages efficiently, and properly deleting messages once they've been processed. By understanding these operations and their slight differences between Standard and FIFO queues, you'll gain a comprehensive skill set that allows for effective message management within AWS's SQS service. Let's embark on this journey to master sending, receiving, and deleting messages in both Standard and FIFO SQS queues.

## Sending Messages to Standard SQS Queues

To send a message to a Standard SQS queue, we initially retrieve the queue and then use the `send_message()` method.

import boto3

sqs = boto3.resource('sqs')
queue = sqs.get_queue_by_name(QueueName='Your-Queue-Name')

response = queue.send_message(
    MessageBody='Hello world!'
)

In addition to basic messages, you can send messages with custom attributes using `MessageAttributes`:

response = queue.send_message(
    MessageBody='Hello with attributes!',
    MessageAttributes={
        'Author': {
            'StringValue': 'John Doe',
            'DataType': 'String'
        },
        'WeeksOnJob': {
            'StringValue': '10',
            'DataType': 'Number'
        }
    }
)

## Sending Messages to FIFO SQS Queues

For sending messages to FIFO queues, the `send_message()` usage is largely the same. However, we need to include an additional parameter: `MessageGroupId`.

response = queue.send_message(
    MessageBody='Hello from FIFO!',
    MessageGroupId='messageGroup1',
    MessageDeduplicationId='1'
)

The `MessageGroupId` parameter is a tag indicating that a group of messages belong together. Messages that belong to the same group are always processed one by one, in a strict order relative to the message group (FIFO). The `MessageDeduplicationId` parameter is used to ensure that messages are not duplicated. It acts as a unique identifier for each message within a specific message group. If a message with the same `MessageDeduplicationId` is sent within the deduplication interval (5 minutes), it will be considered a duplicate and not be delivered again.

## Sending Multiple Messages at a Time

When working with both Standard and FIFO SQS queues, there may be cases where you need to send multiple messages at once efficiently. AWS SQS supports batch sending through the `send_message_batch()` method, allowing up to 10 messages to be sent in a single action. This can significantly reduce the number of network calls to AWS, leading to lower costs and improved throughput.

Here's how you can send multiple messages in one go to a Standard queue:

response = queue.send_message_batch(
    Entries=[
        {
            'Id': 'message1',
            'MessageBody': 'This is the content of message 1'
        },
        {
            'Id': 'message2',
            'MessageBody': 'This is the content of message 2',
            'MessageAttributes': {
                'Author': {
                    'StringValue': 'Jane Doe',
                    'DataType': 'String'
                }
            }
        },
        # You can add up to 10 messages
    ]
)

For FIFO queues, remember to include `MessageGroupId` for each message, and to avoid duplicates, you may also need to specify `MessageDeduplicationId` for each message:

response = queue.send_message_batch(
    Entries=[
        {
            'Id': 'message1',
            'MessageBody': 'This is the content of message 1',
            'MessageGroupId': 'group1',
            'MessageDeduplicationId': '1'
        },
        {
            'Id': 'message2',
            'MessageBody': 'This is the content of message 2',
            'MessageGroupId': 'group1',
            'MessageDeduplicationId': '2'
        },
        # More messages up to 10
    ]
)

Using batch sending effectively lowers the cost by minimizing the number of `send_message` calls and ensures your applications send messages to SQS queues more efficiently.

## Receiving Messages from SQS Queues

Receiving messages from both Standard and FIFO queues utilizes the `receive_message()` method with the queue object. The `MaxNumberOfMessages` parameter allows us to specify the quantity of retrieved messages, anywhere from 1 to a maximum of 10.

messages = queue.receive_messages(
    MaxNumberOfMessages=10
)

By default, the `receive_messages()` method uses short polling and immediately returns a response, even if there are no messages in the queue for that instant. To better manage our queues and avoid empty responses, we should adopt long polling. This solution delays the response until a message arrives in the queue, or the long polling duration ends. Here's how we set up long polling by adding the `WaitTimeSeconds` parameter:

messages = queue.receive_messages(
    MaxNumberOfMessages=10,
    WaitTimeSeconds=20
)

This method waits up to 20 seconds for a message to arrive, returning early if a message becomes available. This efficiently manages the queue, avoiding empty responses.

The response from the `receive_messages` method includes a list of messages, where each message contains various attributes such as the message body (`Body`), a receipt handle (`ReceiptHandle`), and any associated message attributes (`MessageAttributes`) defined when the message was sent.

## Deleting Messages from SQS Queues

Once we have processed a message, we should delete it from the queue to prevent it from being received and processed again. Each message received from the queue comes with a `receipt_handle`, which we use to delete the message. This can be achieved with the message object itself provided by the resource interface.

Here's an example of how to delete a message:

for message in messages:
    # Process your message
    print(message.body)
    # Now that the message is processed, delete it
    message.delete()

This method directly deletes the message using its own delete method, which internally uses its `receipt_handle` to remove it from the queue. This way, the message is not visible in the queue and cannot be received by another consumer.

Keep in mind that if the message is not deleted from the queue before the visibility timeout expires, the message will be available again, and another consumer can process it, potentially leading to the same message being processed multiple times.

You can also batch delete up to 10 messages at once using the `delete_message_batch()` function, but this operation is only possible with the client interface:

response = sqs_client.delete_message_batch(
    QueueUrl='Your-Queue-Url',
    Entries=[
        {
            'Id': 'msg123',
            'ReceiptHandle': 'ReceiptHandle1'
        },
        {
            'Id': 'msg456',
            'ReceiptHandle': 'ReceiptHandle2'
        },
        # More messages up to 10
    ]
)

This can be a more efficient way to delete multiple messages at once, reducing your cost and the number of API requests made.

## Summary

In this lesson, you've learned the similarities and differences between working with Standard and FIFO queues in SQS. Whether it's sending, receiving, or deleting, the operations are quite similar, but with important distinctions in parameters and behaviors. Keep practicing and experimenting with Standard and FIFO queues to reinforce your knowledge.


## Running and Understanding SQS Operations

In this task, your goal is to run a pre-written Python script that employs AWS' Simple Queue Services (SQS) program. The script is designed to create both a standard and a FIFO (First-In-First-Out) queue, then dispatch, receive, and delete various messages within these queues. Make sure to examine the script's structure and commands before executing it for a better understanding of its procedures.

Important Note: Running scripts can modify the resources in our AWS simulator. To revert to the initial state, you can use the reset button located in the top right corner. However, keep in mind that resetting will erase any code changes. To preserve your code during a reset, consider copying it to the clipboard.

```python
import boto3

# Create an SQS resource
sqs = boto3.resource('sqs')

# Create a standard queue
standard_queue = sqs.create_queue(QueueName='test-queue')
standard_queue_url = standard_queue.url

# Create a FIFO queue
fifo_queue = sqs.create_queue(
    QueueName='test-queue.fifo',
    Attributes={
        'FifoQueue': 'true',
        'ContentBasedDeduplication': 'true'
    }
)
fifo_queue_url = fifo_queue.url

# Simple message
response = standard_queue.send_message(
    MessageBody='Hello world!'
)

# Message with custom attributes
response = standard_queue.send_message(
    MessageBody='Hello with attributes!',
    MessageAttributes={
        'Author': {
            'StringValue': 'John Doe',
            'DataType': 'String'
        },
        'WeeksOnJob': {
            'StringValue': '10',
            'DataType': 'Number'
        }
    }
)

# Message with delay seconds
response = standard_queue.send_message(
    MessageBody='Hello with delay!',
    DelaySeconds=10
)

# Message to FIFO queue with message group id and deduplication id
response = fifo_queue.send_message(
    MessageBody='Hello from FIFO queue!',
    MessageGroupId='group1',
    MessageDeduplicationId='abc123'
)

response = standard_queue.receive_messages(
    MaxNumberOfMessages=10  # retrieves up to 10 messages
)

if response:
    for message in response:
        print(f"Processing message: {message.body}")

        # Delete received message from queue
        message.delete()
        print(f"Deleted message: {message.message_id}")
else:
    print("No messages to process!")

```

Here's a breakdown of the provided Python script that interacts with AWS Simple Queue Service (SQS). This script creates both a standard and a FIFO queue, sends messages to them, receives messages, and deletes them. 

### Script Overview

1. **Importing Boto3**: The script starts by importing the `boto3` library, which is the AWS SDK for Python. This library allows Python developers to write software that makes use of services like SQS.

2. **Creating an SQS Resource**: 
   sqs = boto3.resource('sqs')
   This line creates an SQS resource object that allows you to interact with SQS.

3. **Creating a Standard Queue**:
   standard_queue = sqs.create_queue(QueueName='test-queue')
standard_queue_url = standard_queue.url
   This creates a standard queue named `test-queue` and stores its URL for later use.

4. **Creating a FIFO Queue**:
   fifo_queue = sqs.create_queue(
    QueueName='test-queue.fifo',
    Attributes={
        'FifoQueue': 'true',
        'ContentBasedDeduplication': 'true'
    }
)
fifo_queue_url = fifo_queue.url
   This creates a FIFO queue named `test-queue.fifo` with attributes that enable FIFO behavior and content-based deduplication.

5. **Sending Messages to the Standard Queue**:
   - **Simple Message**:
     response = standard_queue.send_message(
    MessageBody='Hello world!'
)
   - **Message with Custom Attributes**:
     response = standard_queue.send_message(
    MessageBody='Hello with attributes!',
    MessageAttributes={
        'Author': {
            'StringValue': 'John Doe',
            'DataType': 'String'
        },
        'WeeksOnJob': {
            'StringValue': '10',
            'DataType': 'Number'
        }
    }
)
   - **Message with Delay**:
     response = standard_queue.send_message(
    MessageBody='Hello with delay!',
    DelaySeconds=10
)

6. **Sending a Message to the FIFO Queue**:
   response = fifo_queue.send_message(
    MessageBody='Hello from FIFO queue!',
    MessageGroupId='group1',
    MessageDeduplicationId='abc123'
)
   This sends a message to the FIFO queue with a specified message group ID and deduplication ID.

7. **Receiving Messages from the Standard Queue**:
   response = standard_queue.receive_messages(
    MaxNumberOfMessages=10  # retrieves up to 10 messages
)
   This retrieves up to 10 messages from the standard queue.

8. **Processing and Deleting Messages**:
   if response:
    for message in response:
        print(f"Processing message: {message.body}")

        # Delete received message from queue
        message.delete()
        print(f"Deleted message: {message.message_id}")
else:
    print("No messages to process!")
   If messages are received, they are processed (printed) and then deleted from the queue to prevent reprocessing.

### Important Notes


- **Resource Modification**: Running this script will create resources in your AWS account. Be cautious, as it may incur costs.
- **Resetting Resources**: If you need to revert to the initial state, use the reset button in your AWS simulator. Remember that this will erase any code changes.
- **Copying Code**: To preserve your code during a reset, copy it to your clipboard before resetting.

### Execution

Before executing the script, ensure that:
- You have the necessary permissions to create and manage SQS queues in your AWS account.
- Your AWS credentials are configured correctly in your environment.

If you are ready to proceed, you can run the script in your Python environment with the appropriate AWS setup.

## Modifying Message Attributes in SQS

In this task, you will modify an existing Python script that interacts with Amazon SQS. The current script creates an SQS queue and sends a message with two attributes: Author and WeeksOnJob. Your task is to add a third attribute, StartingDate, representing the job's starting date in a YYYY-MM-DD format. After implementing your changes, run the script to add the modified message to the queue.

Important Note: Executing scripts can alter the resources in our AWS simulator. To return to the original state, you may use the reset button located in the top right corner. Nonetheless, remember that resetting will eliminate any code modifications. To maintain your code through a reset, consider copying it to the clipboard.

```python
import boto3

# Create an SQS resource
sqs = boto3.resource('sqs')
print("SQS resource created.")

# Create a standard queue
queue = sqs.create_queue(QueueName='test-queue')
print(f"Queue created with URL: {queue.url}")

# Message with custom attributes
response = queue.send_message(
    MessageBody='Hello with attributes!',
    MessageAttributes={
        'Author': {
            'StringValue': 'John Doe',
            'DataType': 'String'
        },
        'WeeksOnJob': {
            'StringValue': '10',
            'DataType': 'Number'
        }
        # TODO: Add a new attribute 'StartingDate' in 'YYYY-MM-DD' format
    }
)
print(f"Message sent with ID: {response['MessageId']}")
```


To modify the existing Python script to include a third attribute, `StartingDate`, representing the job's starting date in a `YYYY-MM-DD` format, you can follow these steps:

1. Add the new attribute to the `MessageAttributes` dictionary.
2. Ensure that the date is formatted correctly.

Here’s the updated script with the new attribute included:

import boto3

# Create an SQS resource
sqs = boto3.resource('sqs')
print("SQS resource created.")

# Create a standard queue
queue = sqs.create_queue(QueueName='test-queue')
print(f"Queue created with URL: {queue.url}")

# Message with custom attributes
response = queue.send_message(
    MessageBody='Hello with attributes!',
    MessageAttributes={
        'Author': {
            'StringValue': 'John Doe',
            'DataType': 'String'
        },
        'WeeksOnJob': {
            'StringValue': '10',
            'DataType': 'Number'
        },
        'StartingDate': {  # New attribute added
            'StringValue': '2025-02-27',  # Example date in YYYY-MM-DD format
            'DataType': 'String'
        }
    }
)
print(f"Message sent with ID: {response['MessageId']}")

### Key Changes Made:
- Added the `StartingDate` attribute to the `MessageAttributes` dictionary.
- Set the `StringValue` of `StartingDate` to an example date (`2025-02-27`).

### Important Notes:
- **Resource Modification**: Running this script will create or modify resources in your AWS account. Be cautious, as it may incur costs.
- **Resetting Resources**: If you need to revert to the initial state, use the reset button in your AWS simulator. Remember that this will erase any code changes.
- **Copying Code**: To preserve your code during a reset, copy it to your clipboard before resetting.

### Execution
Make sure your AWS credentials are configured correctly in your environment, and then run the modified script to add the message with the new attribute to the queue.

## Receiving And Deleting The Messages

In this task, you are given a script that interacts with an existing Amazon Simple Queue Service (SQS) queue called 'MyLearningQueue'. This queue contains motivational messages. Your task is to complete the script by adding functionality to receive a message from the queue, print the content of the message, and then delete it from the queue. Good luck!

Important Note: Running scripts can modify the resources in our AWS simulator. To revert to the initial state, you can use the reset button located in the top right corner. However, keep in mind that resetting will erase any code changes. To preserve your code during a reset, consider copying it to the clipboard.

```python
import boto3

# Create a boto3 resource for SQS
sqs = boto3.resource('sqs')

# Get the existing queue
queue = sqs.get_queue_by_name(QueueName='MyLearningQueue')

# Receive a message from the queue

# Print the received message

# Delete the received message from the queue


```

Here's the completed script that interacts with the existing Amazon SQS queue called `MyLearningQueue`. This script will receive a message from the queue, print its content, and then delete it from the queue.

import boto3

# Create a boto3 resource for SQS
sqs = boto3.resource('sqs')

# Get the existing queue
queue = sqs.get_queue_by_name(QueueName='MyLearningQueue')

# Receive a message from the queue
messages = queue.receive_messages(MaxNumberOfMessages=1)

# Check if any messages were received
if messages:
    for message in messages:
        # Print the received message
        print(f"Received message: {message.body}")

        # Delete the received message from the queue
        message.delete()
        print(f"Deleted message with ID: {message.message_id}")
else:
    print("No messages to process.")

### Key Additions:
1. **Receiving a Message**: The script uses `receive_messages()` to retrieve a message from the queue, specifying `MaxNumberOfMessages=1` to get one message at a time.
2. **Printing the Message**: It checks if any messages were received and prints the content of the message.
3. **Deleting the Message**: After processing the message, it deletes it from the queue using the `delete()` method.

### Important Notes:
- **Resource Modification**: Running this script will modify resources in your AWS account. Be cautious, as it may incur costs.
- **Resetting Resources**: If you need to revert to the initial state, use the reset button in your AWS simulator. Remember that this will erase any code changes.
- **Copying Code**: To preserve your code during a reset, copy it to your clipboard before resetting.

### Execution
Ensure your AWS credentials are configured correctly in your environment, and then run the completed script to receive, print, and delete a message from the `MyLearningQueue`.

## Adding Multiple Messages to SQS FIFO Queue

Good work on your progress! In this task, you'll interact with an AWS SQS FIFO (First-In-First-Out) queue. A FIFO queue named 'student_queue.fifo' is created initially. You're tasked to send two messages to this queue, with each message having the same MessageGroupId of 'CodeSignal2021'. The content of these messages doesn't matter. This task aims to help you learn how to properly send messages to an SQS FIFO queue using Python (boto3).

Important Note: Running scripts can modify the resources in our AWS simulator. To revert to the initial state, you can use the reset button located in the top right corner. However, keep in mind that resetting will erase any code changes. To preserve your code during a reset, consider copying it to the clipboard.

```python
import boto3

sqs = boto3.resource('sqs')

queue = sqs.create_queue(
    QueueName='student_queue.fifo',
    Attributes={
        'FifoQueue': 'true',
        'ContentBasedDeduplication': 'true'
    }
)

# TODO: Send 2 messages to the queue with the same MessageGroupId of 'CodeSignal2021'.

```

Here's the completed script that sends two messages to the SQS FIFO queue named `student_queue.fifo`, using the same `MessageGroupId` of `CodeSignal2021`. 

import boto3

# Create an SQS resource
sqs = boto3.resource('sqs')

# Create the FIFO queue
queue = sqs.create_queue(
    QueueName='student_queue.fifo',
    Attributes={
        'FifoQueue': 'true',
        'ContentBasedDeduplication': 'true'
    }
)

# Send 2 messages to the queue with the same MessageGroupId
message_group_id = 'CodeSignal2021'

# Sending the first message
response1 = queue.send_message(
    MessageBody='This is the first message.',
    MessageGroupId=message_group_id,
    MessageDeduplicationId='msg1'  # Unique ID for deduplication
)

# Sending the second message
response2 = queue.send_message(
    MessageBody='This is the second message.',
    MessageGroupId=message_group_id,
    MessageDeduplicationId='msg2'  # Unique ID for deduplication
)

print(f"First message sent with ID: {response1['MessageId']}")
print(f"Second message sent with ID: {response2['MessageId']}")

### Key Additions:
1. **Sending Messages**: The script sends two messages to the FIFO queue using the `send_message()` method.
2. **MessageGroupId**: Both messages are assigned the same `MessageGroupId` of `CodeSignal2021`.
3. **MessageDeduplicationId**: Each message has a unique `MessageDeduplicationId` to ensure they are treated as distinct messages. This is necessary for FIFO queues to avoid duplicates.

### Important Notes:
- **Resource Modification**: Running this script will create resources in your AWS account. Be cautious, as it may incur costs.
- **Resetting Resources**: If you need to revert to the initial state, use the reset button in your AWS simulator. Remember that this will erase any code changes.
- **Copying Code**: To preserve your code during a reset, copy it to your clipboard before resetting.

### Execution
Ensure your AWS credentials are configured correctly in your environment, and then run the completed script to send the messages to the `student_queue.fifo`.

## Reading and Deleting Messages from an SQS FIFO Queue

Great progress! This task is designed to reinforce your understanding of reading and deleting messages from an AWS SQS FIFO (First-In-First-Out) queue. You'll be implementing Python script which reads and deletes messages from the SQS FIFO queue.

Initially, a FIFO queue named 'student_queue.fifo' is created and 2 messages are sent to this queue. Your task is to:

Receive both messages from the queue in a single call using long polling. Print their contents.
Delete both messages from the queue.
The messages should be processed in the order they were added (First-In-First-Out).

Important Note: Running scripts can modify the resources in our AWS simulator. To revert to the initial state, you can use the reset button located in the top right corner. However, keep in mind that resetting will erase any code changes. To preserve your code during a reset, consider copying it to the clipboard.

```python
import boto3

sqs = boto3.resource('sqs')

queue = sqs.create_queue(
    QueueName='student_queue.fifo',
    Attributes={
        'FifoQueue': 'true',
        'ContentBasedDeduplication': 'true'
    }
)

queue.send_message(
    MessageBody='First Message',
    MessageGroupId='CodeSignal2021'
)

queue.send_message(
    MessageBody='Second Message',
    MessageGroupId='CodeSignal2021'
)

# TODO: Receive both the messages from the queue in a single call using long polling. Print their contents.
# TODO: Delete both messages from the queue.

```

Here's the completed Python script that reads and deletes messages from the SQS FIFO queue named `student_queue.fifo`. This script will receive both messages using long polling, print their contents, and then delete them from the queue.

import boto3

# Create an SQS resource
sqs = boto3.resource('sqs')

# Get the existing FIFO queue
queue = sqs.get_queue_by_name(QueueName='student_queue.fifo')

# Receive both messages from the queue using long polling
messages = queue.receive_messages(
    MaxNumberOfMessages=2,  # Retrieve up to 2 messages
    WaitTimeSeconds=20      # Long polling for up to 20 seconds
)

# Check if any messages were received
if messages:
    for message in messages:
        # Print the received message
        print(f"Received message: {message.body}")

        # Delete the received message from the queue
        message.delete()
        print(f"Deleted message with ID: {message.message_id}")
else:
    print("No messages to process.")

### Key Additions:
1. **Receiving Messages**: The script uses `receive_messages()` to retrieve up to 2 messages from the queue, with a `WaitTimeSeconds` parameter set to 20 for long polling.
2. **Printing the Messages**: It checks if any messages were received and prints the content of each message.
3. **Deleting the Messages**: After processing each message, it deletes them from the queue using the `delete()` method.

### Important Notes:
- **Resource Modification**: Running this script will modify resources in your AWS account. Be cautious, as it may incur costs.
- **Resetting Resources**: If you need to revert to the initial state, use the reset button in your AWS simulator. Remember that this will erase any code changes.
- **Copying Code**: To preserve your code during a reset, copy it to your clipboard before resetting.

### Execution
Ensure your AWS credentials are configured correctly in your environment, and then run the completed script to receive and delete the messages from the `student_queue.fifo`.